# **LAB 4:** Introduction to genetic algorithms

The aim of this practice is to become familiar with the basic ideas behind a **genetic algorithm**. Therefore, we will consider the problem of calculating the inverse of a matrix using genetic algorithms. In general, for any possible matrix, this can be a very complex problem, since the elements of the inverse matrix can be given by any real number.

Therefore, in this first practice, we are going to restrict ourselves to binary matrices (i.e., containing only zeros and ones) and such that their inverses are assumed to exist and to contain only the values in ${-1, 0, 1, 2}$ (although not necessarily all of them).
Let us consider the matrix:

\begin{equation}
A =
    \begin{pmatrix}
        0 & 1 & 0 & 1 & 1 & 0 \\
        1 & 0 & 1 & 0 & 1 & 0 \\
        0 & 1 & 0 & 1 & 0 & 0 \\
        0 & 0 & 1 & 0 & 1 & 1 \\
        1 & 1 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 & 0 & 0
    \end{pmatrix}
\end{equation}

The genetic algorithm to be implemented shall consider the following components:

1. ***ENCODING.*** Determine the representation for each of the population individuals.

In [224]:
# Import necessary libraries
import numpy as np

A = np.array([[0, 1, 0, 1, 1, 0],
              [1, 0, 1, 0, 1, 0],
              [0, 1, 0, 1, 0, 0],
              [0, 0, 1, 0, 1, 1],
              [1, 1, 0, 1, 0, 0],
              [0, 0, 0, 1, 0, 0]])

def encode_string(matrix):
    encoded_string = ""
    encoding_scheme = {-1:'-1', 0: '0', 1: '1', 2: '2'}
    for row in matrix:
        for element in row:
            encoded_string += encoding_scheme[element]
    return encoded_string


encoded_A = encode_string(A)
print(encoded_A)

010110101010010100001011110100000100


2. ***FITNESS FUNCTION.*** Define the function to be used.

In [225]:
import numpy as np

def fitness(A, population):
    fitness_scores = np.zeros(len(population), dtype=int)

    for k, individual in enumerate(population):
        try:
            inverse_A = np.linalg.inv(A)
            inverse_individual = np.linalg.inv(individual)

            identity_matrix = np.eye(len(A))
            fitness_score = np.sum(np.isclose(inverse_A * inverse_individual, identity_matrix))
            fitness_scores[k] = fitness_score
        except np.linalg.LinAlgError as e:
            fitness_scores[k] = 0

    return fitness_scores


3. ***INITIAL POPULATION.*** Consider a population of 10 individuals (i.e. 10 matrices of dimension $6\times 6$ whose values are in $\{-1, 0, 1, 1, 2\}$, randomly initialised).

In [226]:
val_min, val_max = -1, 2
num_population = 10

N = 6
population = np.random.randint(low=val_min,
                               high=val_max+1,
                               size=(num_population, N, N))

print(fitness(A, population))

[20 18 18 16 17 16 18 16 19 18]


4. ***PARENT SELECTION.*** Select a method for choosing progenitors in order to create the new individuals. This function must make use of the previously defined fitness function.

In [227]:
import random

def progenitorSelection(population): #ROULETTE METHOD
  probabilities = np.zeros(len(population))
  fit = fitness(A, population)
  sumaProbabilities = np.sum(fit)
  for i in range(len(fit)):
      probabilities[i] = max(fit[i] - (np.mean(fit) - np.std(fit)*2),0)
  sumaProbabilities = 0
  p = random.random()
  for i in range(len(probabilities)):
    sumaProbabilities += probabilities[i]
    if p <= sumaProbabilities:
      return i
  return 0

print(progenitorSelection(population))

0


5. ***CROSSOVER METHOD.*** Set a strategy for crossover matrices (e. g., one-point crossover or two-point crossover). Crossover probability: 0.95.

In [228]:
def crossover(chromosome1, chromosome2, prob_cross):
  if random.random() <= prob_cross:
    randomIndex = random.randint(0, len(chromosome1)*len(chromosome1) - 1)
    cross_chromosome1=[]
    cross_chromosome2=[]
    for i in range(len(chromosome1)):
      row1=[]
      row2=[]
      for j in range(len(chromosome2)):
        if i*len(chromosome1) + j < randomIndex:
          row1.append(chromosome1[i][j])
          row2.append(chromosome2[i][j])
        else:
          row1.append(chromosome2[i][j])
          row2.append(chromosome1[i][j])
      cross_chromosome1.append(row1)
      cross_chromosome2.append(row2)
    return cross_chromosome1, cross_chromosome2
  else:
    return chromosome1, chromosome2


print(population[0])
print(population[1])
cross_chromosome1, cross_chromosome2 = crossover(population[0], population[1], 0.95)
print(cross_chromosome1)
print(cross_chromosome2)

[[ 0 -1  0  0  0  0]
 [ 1 -1  0 -1  1  2]
 [ 2  0  0 -1 -1  1]
 [ 0 -1 -1 -1  1  2]
 [ 1 -1 -1  2  2  2]
 [ 1 -1 -1  1  1 -1]]
[[ 1  2 -1  2  0  0]
 [ 0  1 -1  0  0  0]
 [ 2 -1 -1 -1  2  2]
 [-1  0 -1  0  2 -1]
 [-1  1  0  1  1  0]
 [ 2  0  1  0  1  2]]
[[0, -1, 0, 0, 0, 0], [1, -1, 0, -1, 1, 2], [2, 0, 0, -1, -1, 1], [0, -1, -1, 0, 2, -1], [-1, 1, 0, 1, 1, 0], [2, 0, 1, 0, 1, 2]]
[[1, 2, -1, 2, 0, 0], [0, 1, -1, 0, 0, 0], [2, -1, -1, -1, 2, 2], [-1, 0, -1, -1, 1, 2], [1, -1, -1, 2, 2, 2], [1, -1, -1, 1, 1, -1]]


6. ***MUTATION METHOD.*** Set a mutation mechanism. For example, each gen mutates with a probability of $p=0.01$. Then, in each iteration, approximately a 1\% of genes will mutate.

In [229]:
def mutation(chromosome, prob_mut):
    mutated_chromosome = chromosome
    for i in range(len(chromosome)):
        for j in range(len(chromosome[i])):
            if random.random() <= prob_mut:
              mutated_chromosome[i][j] = random.uniform(-1, 2)
    return mutated_chromosome


mutate_chromosome1 = mutation(population[0], 0.01)
print(population[0])
print(mutate_chromosome1)

[[ 0 -1  0  0  0  0]
 [ 1 -1  0 -1  1  2]
 [ 2  0  0 -1 -1  1]
 [ 0 -1 -1 -1  1  2]
 [ 1  0 -1  2  2  2]
 [ 1 -1 -1  1  1 -1]]
[[ 0 -1  0  0  0  0]
 [ 1 -1  0 -1  1  2]
 [ 2  0  0 -1 -1  1]
 [ 0 -1 -1 -1  1  2]
 [ 1  0 -1  2  2  2]
 [ 1 -1 -1  1  1 -1]]


7. ***SURVIVAL SELECTION OR SUBSTITUTION.*** Establish a method for selecting which individuals will be part of the next population. Take into account both the individuals of the existing population as well as the new ones generated by crossover and which may have mutated.

In [230]:
import pdb
def survivorSelection(population, new_individuals): #generational model (same size population)
  #pdb.set_trace()
  bigValue = 1000
  lowValue = -1
  fitPopulation = fitness(A,population)
  fitNewIndividuals = fitness(A,new_individuals)
  sizeOfNewIndividuals = len(new_individuals)
  indexToEliminate = []
  newPopulation = []
  for i in range(sizeOfNewIndividuals):
    if np.min(fitPopulation) < np.max(fitNewIndividuals):
      maxIndexNewIndividuals = np.argmax(fitNewIndividuals)
      minIndexPopulation = np.argmin(fitPopulation)
      fitPopulation[minIndexPopulation] = bigValue
      fitNewIndividuals[maxIndexNewIndividuals] = lowValue
      newPopulation.append(new_individuals[maxIndexNewIndividuals])
      indexToEliminate.append(minIndexPopulation)
  for i in range(len(population)):
    if i not in indexToEliminate:
      newPopulation.append(population[i])
  return newPopulation

val_min, val_max = -1, 2
num_population = 2
N = 6
new_individuals = np.random.randint(low=val_min,
                               high=val_max+1,
                               size=(num_population, N, N))
population = np.random.randint(low=val_min,
                               high=val_max+1,
                               size=(num_population, N, N))
#print(population)
print(new_individuals)
newPopulation = survivorSelection(population, new_individuals)
#print(newPopulation)

[[[ 0  1  2  0  0 -1]
  [ 0  0 -1 -1  2  1]
  [ 0 -1  0 -1  1 -1]
  [ 0  0 -1 -1  1  0]
  [ 1  0  0  0  2 -1]
  [ 1  0  2  0 -1 -1]]

 [[ 1 -1 -1  0  0  1]
  [ 0 -1 -1  1  2  2]
  [ 1  0 -1  2  1  0]
  [ 0 -1  1 -1  0  0]
  [ 0  2 -1  1  2  2]
  [ 1  1 -1  1  2  2]]]


### ***MAIN LOOP***

The genetic algorithm must stop, either when it finds a solution, or when it has reached a maximum number of iterations (e.g. $200$). In the latter case, the algorithm must return the individual with the best {fitness} among the population that exists at the time of stopping the execution.
For parent selection, use the roulette method with generational replacement (i.e. descendants replace parents in the population).

In [233]:
#import pdb
#pdb.set_trace()
prob_cross = 0.95
prob_mut = 0.01

num_iterations = 200

A = np.array([[0, 1, 0, 1, 1, 0],
              [1, 0, 1, 0, 1, 0],
              [0, 1, 0, 1, 0, 0],
              [0, 0, 1, 0, 1, 1],
              [1, 1, 0, 1, 0, 0],
              [0, 0, 0, 1, 0, 0]])

val_min, val_max = -1, 2
num_population = 10
N = 6
population = np.random.randint(low=val_min,
                               high=val_max+1,
                               size=(num_population, N, N))
numberOfParents = 4
maxFit = N*N
parents = []
for iter in range(num_iterations):
  ## Parent selection
  parents = []
  for i in range(numberOfParents):
    index = progenitorSelection(population)
    parents.append(population[index])
  ## Crossover
  cross = []
  for i in range(int(len(parents)/2)):
    chromosome, chromosome2 = crossover(parents[i], parents[i+1], prob_cross)
    cross.append(chromosome)
    cross.append(chromosome2)
  ## Mutation
  mutations = []
  for i in range(len(cross)):
    mutation1 = mutation(cross[i], prob_mut)
    mutations.append(mutation1)
  population = survivorSelection(population, mutations)
  fit = fitness(A,population)
  print(fit)
  for i in range(len(fit)):
    if fit[i] == maxFit:
      print(population[i])
      break
#print(len(population))



[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 16 16 20 18 17 17 20 16]
[16 16 1

In [232]:
"""prob_cross = 1
prob_mut = 0.01

num_iterations = 200

A = np.array([[0, 1, 0, 1, 1, 0],
              [1, 0, 1, 0, 1, 0],
              [0, 1, 0, 1, 0, 0],
              [0, 0, 1, 0, 1, 1],
              [1, 1, 0, 1, 0, 0],
              [0, 0, 0, 1, 0, 0]])

val_min, val_max = -1, 2
num_population = 10
N = 6
population = np.random.randint(low=val_min,
                               high=val_max+1,
                               size=(num_population, N, N))
numberOfParents = 4
parents = []
childs = []
maxFit = len(A)*len(A)


for iter in range(num_iterations):
  fit = fitness(A, population)
  if maxFit in fit: #inverse
    print(population[np.where(fitness == maxFit)[0]])
    break
  ## Parent selection
  parent1 = population[progenitorSelection(population)]
  parent2 = population[progenitorSelection(population)]
  parent3 = population[progenitorSelection(population)]
  parent4 = population[progenitorSelection(population)]
  ## Crossover
  cross_chromosome1, cross_chromosome2 = crossover(parent1, parent2, prob_cross)
  cross_chromosome3, cross_chromosome4 = crossover(parent3, parent4, prob_cross)
  ## Mutation
  mutation1 = mutation(cross_chromosome1, prob_mut)
  mutation2 = mutation(cross_chromosome2, prob_mut)
  mutation3 = mutation(cross_chromosome3, prob_mut)
  mutation4 = mutation(cross_chromosome4, prob_mut)
  mutations = [mutation1, mutation2, mutation3, mutation4]
  ## Survivor selection
  population = survivorSelection(population, mutations)
print(population)"""


'prob_cross = 1\nprob_mut = 0.01\n\nnum_iterations = 200\n\nA = np.array([[0, 1, 0, 1, 1, 0],\n              [1, 0, 1, 0, 1, 0],\n              [0, 1, 0, 1, 0, 0],\n              [0, 0, 1, 0, 1, 1],\n              [1, 1, 0, 1, 0, 0],\n              [0, 0, 0, 1, 0, 0]])\n\nval_min, val_max = -1, 2\nnum_population = 10\nN = 6\npopulation = np.random.randint(low=val_min,\n                               high=val_max+1,\n                               size=(num_population, N, N))\nnumberOfParents = 4\nparents = []\nchilds = []\nmaxFit = len(A)*len(A)\n\n\nfor iter in range(num_iterations):\n  fit = fitness(A, population)\n  if maxFit in fit: #inverse\n    print(population[np.where(fitness == maxFit)[0]])\n    break\n  ## Parent selection\n  parent1 = population[progenitorSelection(population)]\n  parent2 = population[progenitorSelection(population)]\n  parent3 = population[progenitorSelection(population)]\n  parent4 = population[progenitorSelection(population)]\n  ## Crossover\n  cross_chro